In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier as RFC_
from sklearn.ensemble import RandomForestRegressor as RFR_
from sklearn import metrics
import time
from pandas.tseries.offsets import DateOffset

In [ ]:
def test(imp_fert,md_c,md_r,uf_list,nick_list,lim_active,feat_code,quanti_col,anos_teste):

    #i=time.time()
    
    RFC=RFC_(max_depth=md_c)

    RFR=RFR_(max_depth=md_r)

    # Filtra UF e produtos de interesse específico
    imp_fert=imp_fert[imp_fert.SG_UF_NCM.isin(uf_list)]
    imp_fert=imp_fert[imp_fert.nick.isin(nick_list)]

    # Identifica colunas categóricas
    cat_cols=['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF']
    imp_fert[cat_cols]=imp_fert[cat_cols].astype('category')
    
    
    # Filtra deixando passar somente categorias ativas nos últimos anos
    filtro=\
    imp_fert[imp_fert.CO_ANO>imp_fert.CO_ANO.max()-lim_active]\
    [['nick','SG_UF_NCM','CO_PAIS','CO_URF']].drop_duplicates()
    imp_fert=imp_fert.merge(filtro)

    # Elimina categorias únicas
    if len(imp_fert.SG_UF_NCM.unique())==1:
        imp_fert.drop(columns='SG_UF_NCM',inplace=True)
    if len(imp_fert.nick.unique())==1:
        imp_fert.drop(columns='nick',inplace=True)
    


    # Salva as datas de início e fim dos registros
    imp_fert.insert(0,'date',pd.to_datetime(imp_fert.CO_ANO.astype(str)+'/'+imp_fert.CO_MES.astype(str)+'/1'))    
    date_min=imp_fert.date.min()
    date_max=imp_fert.date.max()

    

    # Funde e codifica características
    a=0
    for col in feat_code:
        if col in imp_fert.columns:
                
            if a==0:
                imp_fert['code']=imp_fert[col].astype(str)
                imp_fert.drop(columns=[col],inplace=True)
                a+=1
            else:
                imp_fert['code']='-'+imp_fert['code'].astype(str)+'-'+imp_fert[col].astype(str)
                imp_fert.drop(columns=[col],inplace=True)
                
    if len(feat_code)>0:
        imp_fert.code=imp_fert.code.astype('category')
    
    # Matriz Esparsa
    %run '~/git/BD/Comexstat/matriz_esparsa.ipynb'
    imp_fert=\
    imp_fert.groupby(by=imp_fert.select_dtypes(include='category').columns.tolist()+['CO_ANO'],observed=False)[quanti_col].sum().reset_index()
    
    # Corrige bordas da matriz esparsa
    imp_fert['date']=pd.to_datetime(imp_fert.CO_ANO.astype(str)+'/'+imp_fert.CO_MES.astype(str)+'/1')
    imp_fert=imp_fert[(imp_fert.date>=date_min)&(imp_fert.date<=date_max)]

    # Diferencia ocorrências reais de não ocorrência
    imp_fert=imp_fert.assign(oc=0)
    imp_fert.loc[imp_fert.KG_LIQUIDO>0,'oc']=1

    
    # Cria df para coletar os resultados do loop
    q=pd.DataFrame()
    r=pd.DataFrame()
    
    # Inicia loop sobre as datas
    for date in imp_fert.date.unique():
        
        # Abre contagem de tempo para o loop externo
        i_p=time.time()
        
        # Filtra deixando passar registros menores ou iguais a data do loop
        imp_fert_t=imp_fert[(imp_fert.date<=date)]
    
        # Certifica duração do período de estudo
        if imp_fert_t.date.max()-DateOffset(years=anos_teste)>=imp_fert_t.date.min():

            imp_fert_t=imp_fert_t[imp_fert_t.date>imp_fert_t.date.max()-DateOffset(years=anos_teste)]

            # Adequa formato da data para processamento ML
            imp_fert_t['date']=imp_fert_t.date.astype(int)
    
    
            # ******* CLASSIFCAÇÃO *******
    
            # Transforma categoricos em dummies
            imp_fert_dumm=pd.get_dummies(imp_fert_t,prefix_sep='~')
            
            # Identifica registros para treino e teste
            train_dumm=imp_fert_dumm[imp_fert_dumm.date< imp_fert_dumm.date.max()]
            test_dumm =imp_fert_dumm[imp_fert_dumm.date==imp_fert_dumm.date.max()]
            
    
            # Identifica X e y classifier
            X_train_c=train_dumm.drop(columns=[quanti_col,'oc'])
            y_train_c=train_dumm.oc
            
            X_test_c =test_dumm.drop(columns=[quanti_col,'oc'])
            y_test_c =test_dumm.oc
            
            
            # Fit classifier
            RFC.fit(X_train_c,y_train_c)
            # Predict classifier
            test_pred_c=X_test_c.assign(oc_pred=RFC.predict(X_test_c)).assign(oc=y_test_c)

            #display('\n','test_pred_c',test_pred_c.head(1))
    
            q=pd.concat([q,test_pred_c])
    
            
            
            # REGRESSÃO
    
            imp_fert_t=imp_fert_t[imp_fert_t.oc==1]
    
            if len(imp_fert_t[imp_fert_t.date.astype('datetime64[ns]')==date])>0:
        
                # Transforma categoricos em dummies
                imp_fert_dumm=pd.get_dummies(imp_fert_t,prefix_sep='~')
                
                # Identifica registros para treino e teste
                train_dumm=imp_fert_dumm[imp_fert_dumm.date< imp_fert_dumm.date.max()]
                test_dumm =imp_fert_dumm[imp_fert_dumm.date==imp_fert_dumm.date.max()]
        
                # Identifica X e y
                X_train_r=train_dumm.drop(columns=[quanti_col,'oc'])
                y_train_r=train_dumm.KG_LIQUIDO
                
                X_test_r =test_dumm.drop(columns=[quanti_col,'oc'])
                y_test_r =test_dumm.KG_LIQUIDO
                
                # Fit regressor
                RFR.fit(X_train_r,y_train_r)
                
                # Predict classifier
                test_pred_r=X_test_r.assign(kg_pred=RFR.predict(X_test_r)).assign(KG_LIQUIDO=y_test_r)
    
                r=pd.concat([r,test_pred_r])
            
        
        f_p=time.time()
        #print(f"Date {date.date()} : Parcial:{round((f_p-i_p)/60,3)} minutos.")

    #f=time.time()
    #d=f-i
    #print(f"Duração de {round(d/60)} minutos.")

    def f_dumm(df_dumm):
        a=pd.from_dummies(df_dumm.select_dtypes(include='bool'),sep='~')
        b=df_dumm.select_dtypes(exclude='bool')
        c=pd.concat([a,b],axis=1)
        return c


    
    q=f_dumm(q)
    
    r=f_dumm(r)
    if len(q)>0:
        if len(r)>0:
            q=q.merge(r,how='outer')

    return q

In [ ]:
print('\n*** test(imp_fert,md_c,md_r,uf_list,nick_list,lim_active,feat_code,quanti_col,anos_teste) ***\n')